In [35]:
import math
from statistics import mean
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

PI = math.pi

In [36]:

 
def findU(cn, n, s):
    # Function f(u)
    def f(u):
        return u + cn * math.sin(2 * n * u) - s

    # Derivative of f(u) with respect to u
    def df(u):
        return 1 + 2 * n * cn * math.cos(2 * n * u)

    u = s  # Initial guess
    tolerance = 1e-8  # Tolerance for the solution
    maxIterations = 1000  # Maximum number of iterations
    iteration = 0
    error = 10 * tolerance  # Initialize error

    # Newton-Raphson method
    while error > tolerance and iteration < maxIterations:
        uNext = u - f(u) / df(u)  # Next approximation
        error = abs(uNext - u)  # Update error
        u = uNext  # Update u for the next iteration
        iteration += 1

    if iteration == maxIterations:
        print("Solution did not converge")
        return None

    #print(f(u))
    return u


def animation_data(n, N, j):
    t = j
    c = 1

    hx = []
    hy = []
    hz = []

    rx = []
    ry = []
    rz = []

    kappa0 = [0] * (N + 1)

    cosGamma = n / 2 - math.sqrt(n*n - 4) / 2
    sinGamma = math.sqrt(1 - cosGamma*cosGamma)

    # Initial rotation angle calculation
    an = n * math.sqrt(n*n - 4)
    bn = (n*n - 4 - an) / n / 2.0 
    cn =  (math.sqrt(n*n-4)-n)/(2*PI*n*n)
    norm_r = 1/(PI*an)

    i = 0
    xi = 0 - c * t
    #print(f"the value of traveling coordinate is {xi}")
    #print(f"the value of arclength coordinate is {i}")

    cos2s = math.cos(2 * xi)
    sin2s = math.sin(2 * xi)
    cosns = math.cos(n * xi)
    sinns = math.sin(n * xi)

    cosn_m_s = math.cos(2 * (n - 1) * xi)
    cosn_p_s = math.cos(2 * (n + 1) * xi)

    sinn_m_s = math.sin(2 * (n - 1) * xi)
    sinn_p_s = math.sin(2 * (n + 1) * xi)

    tempBx = cos2s * cosns * cosGamma + sin2s * sinns
    tempBy = sin2s * cosns * cosGamma - cos2s * sinns
    hz.append(cosns * sinGamma)

    tempRx = sinGamma / 2 * ((n - cosGamma) * sin2s - (1 + cosGamma) * sinn_m_s / (2*n - 2) + (1 - cosGamma) * sinn_p_s / (2*n + 2))
    tempRy = -sinGamma / 2 * ((n - cosGamma) * cos2s + (1 + cosGamma) * cosn_m_s / (2*n - 2) + (1 - cosGamma) * cosn_p_s / (2*n + 2))
    rz.append(-1 * cosns * sinns * sinGamma * sinGamma / n *norm_r)

    norm1 = math.sqrt(tempRx**2 + tempRy**2)
    cosths_0 = tempRy / norm1
    sinths_0 = tempRx / norm1

    tempRx = tempRx*norm_r
    tempRy = tempRy*norm_r

    rx.append(cosths_0 * tempRx - sinths_0 * tempRy)
    ry.append(sinths_0 * tempRx + cosths_0 * tempRy)

    hx.append(cosths_0 * tempBx - sinths_0 * tempBy)
    hy.append(sinths_0 * tempBx + cosths_0 * tempBy)

    for i in range(1,N + 1):
        s = i / N * PI

        if N < 15:
            u = findU(cn, n, s)  # Assuming findU is defined elsewhere
        else:
            u = s

        u = s
        xi = u - c * t
        #print(f"the value of traveling coordinate is {xi}")
        #print(f"the value of arclength coordinate is {i}")

        cos2s = math.cos(2 * xi)
        sin2s = math.sin(2 * xi)
        cosns = math.cos(n * xi)
        sinns = math.sin(n * xi)

        cosn_m_s = math.cos(2 * (n - 1) * xi)
        cosn_p_s = math.cos(2 * (n + 1) * xi)

        sinn_m_s = math.sin(2 * (n - 1) * xi)
        sinn_p_s = math.sin(2 * (n + 1) * xi)

        tempBx = cos2s * cosns * cosGamma + sin2s * sinns
        tempBy = sin2s * cosns * cosGamma - cos2s * sinns
        hz.append(cosns * sinGamma)

        tempRx = sinGamma / 2 * ((n - cosGamma) * sin2s - (1 + cosGamma) * sinn_m_s / (2*n - 2) + (1 - cosGamma) * sinn_p_s / (2*n + 2))
        tempRy = -sinGamma / 2 * ((n - cosGamma) * cos2s + (1 + cosGamma) * cosn_m_s / (2*n - 2) + (1 - cosGamma) * cosn_p_s / (2*n + 2))
       
        tempRx = tempRx*norm_r
        tempRy = tempRy*norm_r
        rz.append(-1 * cosns * sinns * sinGamma * sinGamma / n * norm_r)

      

        rx.append(cosths_0 * tempRx - sinths_0 * tempRy)
        ry.append(sinths_0 * tempRx + cosths_0 * tempRy)

        hx.append(cosths_0 * tempBx - sinths_0 * tempBy)
        hy.append(sinths_0 * tempBx + cosths_0 * tempBy)

  

    return hx, hy, hz, rx, ry, rz

def edge_and_mid_length(hx,hy,hz,rx,ry,rz):
    N = len(hx)-1
    dl_e1 = np.zeros(N)
    dl_e2 = np.zeros(N)
    dl_r  = np.zeros(N)
    dl_b  = np.zeros(N)
    l = .2

    for i in range(N):
        dl_r[i] = ((rx[i]-rx[i+1])**2 + (ry[i]-ry[i+1])**2 + (rz[i]-rz[i+1])**2)
        dl_b[i] = ((hx[i]-hx[i+1])**2 + (hy[i]-hy[i+1])**2 + (hz[i]-hz[i+1])**2)

    return dl_r,dl_b

def plot2D(path_rx,path_ry, label):
       

   trace = go.Scatter(x=path_rx, y=path_ry,
                         mode='markers',
                         line=dict(width=4),
                         name = label)
   return trace




def plot3D(path_rx,path_ry,path_rz,label):
       
       
       
      indices = np.arange(len(path_rx))
    
      # Convert the indices array to a string array and format it
      hover_text = [f'Index: {i}<br>X: {x}<br>Y: {y}<br>Z: {z}' 
                  for i, x, y, z in zip(indices, path_rx, path_ry, path_rz)]
    
      trace = go.Scatter3d(
         x=path_rx, 
         y=path_ry, 
         z=path_rz,
         mode='lines+markers',  # Specify both lines and markers
        line=dict(
            width=1,
            dash='dot'  # Dotted line
        ),
        marker=dict(
            symbol='circle',  # Spherical markers
            size=2  # Adjust size as needed
        ),
         hovertext=hover_text,  # Set the custom hover text
         hoverinfo='text',  # Display custom text only
         name=label
      )
    
      return trace

In [51]:
n  = 3;
N  =  n ;
 
t = np.linspace(0,np.pi,100);
u = np.linspace(0,N-1,N )
len_r = np.zeros(len(t))
len_edge1= np.zeros(len(t))
len_edge2= np.zeros(len(t))

fig = go.Figure()
for i in range(10):
    ti =  t[i]
    hx, hy, hz, rx, ry, rz = animation_data(n,N,ti)
    dl_r,dl_b = edge_and_mid_length(hx, hy, hz, rx, ry, rz)

    trace = plot2D(u,dl_b,"")
    fig.add_trace(trace)
    

fig.show()    

    

    



In [52]:
print(dl_b)

[2.09882587 2.09882587 2.09882587]


In [39]:
fig = go.Figure()
fig.add_trace(plot3D(rx,ry,rz,""))
fig.show()

In [40]:
for i in range(len(hx)):
    print(hx[i]**2 + hy[i]**2 + hz[i]**2)

0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999


In [41]:
rx

[0.0, -0.05774100389239544, 0.05774100389239542, 1.200866100316464e-17]

In [42]:
ry

[0.06667356828110747,
 -0.033336784140553716,
 -0.03333678414055375,
 0.06667356828110747]

In [43]:
rz

[-0.0, 1.654410162261043e-18, 3.308820324522086e-18, 4.963230486783128e-18]